# Loan Prediction

This is my first kernel, (completely) copied for learning purposes from:

https://www.kaggle.com/yaheaal/loan-status-with-different-models

Other kernels I got inspiration from:

https://www.kaggle.com/aarti19/loan-predictionproblem
https://www.kaggle.com/yonatanrabinovich/loan-prediction-dataset-ml-project

Thanks @Yaheaal, @aarti19, @yonatanrabinovich

Libraries: **sklearn, matplotlib, numpy, pandas, seaborn, scipy**


Missing values filled in using **backward 'bfill' method** for numerical columns , and **most frequent value** for categorical columns (simple techniques)


* Train models

    **a) Logistic regression**
    
    **b) KNeighborsClassifier**
    
    **C) SVC**
    
    **d) DecisionTreeClassifier**


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os #paths to file
import numpy as np # linear algebra
import pandas as pd # data processing
import warnings# warning filter


#ploting libraries
import matplotlib.pyplot as plt 
import seaborn as sns

#relevant ML libraries
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

#ML models
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

#default theme
sns.set(context='notebook', style='darkgrid', palette='deep', font='sans-serif', font_scale=1, color_codes=False, rc=None)

#warning hadle
warnings.filterwarnings("ignore")

print("Libraries imported")

In [ ]:
df = pd.read_csv('../input/loan-prediction-problem-dataset/train_u6lujuX_CVtuZ9i.csv')

* # **1. Exploratory Analysis**

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
df.info()
# We have missing data , we will handle them as we go

In [ ]:
# Describe the numerical data
df.describe()

In [ ]:
# We will change the type of Credit_History to object becaues we can see that it is 1 or 0
df['Credit_History'] = df['Credit_History'].astype('O')

# describe categorical data ("object")
df.describe(include='O')

In [ ]:
# We will drop ID because it's not important for our model and it will just mislead the mode

df.drop('Loan_ID', axis=1, inplace=True)

In [ ]:
df.duplicated().any()

# We got no duplicated rows

In [ ]:
# Let's look at the target percentage

plt.figure(figsize=(8,6))
sns.countplot(df['Loan_Status']);

print('The percentage of Y class : %.2f' % (df['Loan_Status'].value_counts()[0] / len(df)))
print('The percentage of N class : %.2f' % (df['Loan_Status'].value_counts()[1] / len(df)))

# We can consider it as imbalanced data, but for now i will not

In [ ]:
df.columns

In [ ]:
df.head(1)

In [ ]:
# Credit_History

grid = sns.FacetGrid(df,col='Loan_Status', size=3.2, aspect=1.6)
grid.map(sns.countplot, 'Credit_History');

# We didn't give a loan for most people who got Credit History = 0
# but we did give a loan for most of people who got Credit History = 1
# so we can say if you got Credit History = 1 , you will have better chance to get a loan

In [ ]:
print(pd.crosstab(df['Credit_History'],df['Loan_Status']))

df_history_Y = df[df['Credit_History'] == 1]
df_history_N = df[df['Credit_History'] == 0]

perc_df_self_Y = df_history_Y['Loan_Status'].value_counts()['Y']/len(df_history_Y)
perc_df_self_N = df_history_N['Loan_Status'].value_counts()['Y']/len(df_history_N)

print('\n')

print('Percentage loans with Credit_History Y: %.3f' %perc_df_self_Y)
print('Percentage loans with Credit_History N: %.3f' %perc_df_self_N)

In [ ]:
# Gender

grid = sns.FacetGrid(df,col='Loan_Status', size=3.2, aspect=1.6)
grid.map(sns.countplot, 'Gender');

# Most males got loan and most females got one too so (No pattern)

# I think it's not so important feature, we will see later

In [ ]:
# Married
plt.figure(figsize=(15,5))
sns.countplot(x='Married', hue='Loan_Status', data=df);

# Most people who get married did get a loan.
# If you'r married then you have better chance to get a loan.
# Good feature

In [ ]:
# Dependents

plt.figure(figsize=(15,5))
sns.countplot(x='Dependents', hue='Loan_Status', data=df);

# First if Dependents = 0 , we got higher chance to get a loan ((very hight chance))
# Good feature

In [ ]:
# Education

grid = sns.FacetGrid(df,col='Loan_Status', size=3.2, aspect=1.6)
grid.map(sns.countplot, 'Education');

# If you are graduated or not, you will get almost the same chance to get a loan (No pattern)
# Here you can see that most people did graduated, and most of them got a loan.
# On the other hand, most of people who did't graduate also got a loan, but with less 
# percentage from people who graduated.

In [ ]:
df_graduated = df[df['Education'] == 'Graduate']
df_not_graduated = df[df['Education'] != 'Graduate']

print('Graduate')
df_graduated.head()

In [ ]:
print('Not graduate')
df_not_graduated.head()

In [ ]:
n_loans_gr = len(df_graduated[df_graduated['Loan_Status'] == 'Y'])
#n_loans_gr = df_graduated['Loan_Status'].value_counts()[0]
length_gr =len(df_graduated)
perc_df_graduated_Y = n_loans_gr/length_gr

n_loans_not_gr = len(df_not_graduated[df_not_graduated['Loan_Status'] == 'Y'])
#n_loans_gr = df_graduated['Loan_Status'].value_counts()[0]
length_not_gr =len(df_not_graduated)
perc_df_not_graduated_Y = n_loans_not_gr/length_not_gr

print('Percentage loans for NOT graduated: %.2f' % perc_df_not_graduated_Y)

print('Percentage loans for graduated: %.2f' % perc_df_graduated_Y)

In [ ]:
# Self_Employed

grid = sns.FacetGrid(df,col='Self_Employed', size=3.2, aspect=1.6)
grid.map(sns.countplot, 'Loan_Status');

# No pattern (same as Education)

In [ ]:
df_self_employed_Y = df[df['Self_Employed'] == 'Yes']
df_self_employed_N = df[df['Self_Employed'] == 'No']

n_loans_self_y = df_self_employed_Y['Loan_Status'].value_counts()[0]
length_self_y =len(df_self_employed_Y)
perc_df_self_Y = n_loans_self_y/length_self_y

n_loans_self_n = df_self_employed_N['Loan_Status'].value_counts()[0]
length_self_n =len(df_self_employed_N)
perc_df_self_N = n_loans_self_n/length_self_n

print('Percentage loans for self employed: %.2f' % perc_df_self_Y)

print('Percentage loans for not self employed: %.2f' % perc_df_self_N)

In [ ]:
# Property_Area

plt.figure(figsize=(15,5))
sns.countplot(x='Property_Area', hue='Loan_Status', data=df);

# We can say, Semiurban Property_Area got more than 50% chance to get a loan
# Good feature

In [ ]:
# ApplicantIncome

plt.scatter(df['ApplicantIncome'], df['Loan_Status']);

# No pattern

In [ ]:
# The numerical data

df.groupby('Loan_Status').median() # median because Not affected with outliers

# We can see that when we got low median in CoapplicantInocme we got Loan_Status = N

# CoapplicantInocme is a good feature

# **2. Data Cleaning**

In [ ]:
df.isnull().sum().sort_values(ascending=False)

In [ ]:
# We will separate the numerical columns from the categorical

cat_data = []
num_data = []

for i,c in enumerate(df.dtypes):
    if c == object:
        cat_data.append(df.iloc[:, i])
    else :
        num_data.append(df.iloc[:, i])
        

cat_data = pd.DataFrame(cat_data).transpose()
num_data = pd.DataFrame(num_data).transpose()

In [ ]:
cat_data.head()

In [ ]:
num_data.head()

In [ ]:
cat_data.isnull().sum().any() #cat_data missing values?

In [ ]:
# Categorical data
# If you want to fill every column with its own most frequent value you can use.

cat_data = cat_data.apply(lambda x:x.fillna(x.value_counts().index[0]))
cat_data.isnull().sum().any() # No more missing data

In [ ]:
num_data.isnull().sum().any() # num_data missing data?

In [ ]:
# Numerical data
# Fill every missing value with their previous value in the same column.

num_data.fillna(method='bfill', inplace=True)
num_data.isnull().sum().any() # no more missing data

In [ ]:
from sklearn.preprocessing import LabelEncoder  
le = LabelEncoder()
cat_data.head()

In [ ]:
# Transform the target column

target_values = {'Y': 0 , 'N' : 1}

# Save 'Loan_Status' column in 'target'
target = cat_data['Loan_Status']

#Remove 'Loan_Status' column from cat_data
cat_data.drop('Loan_Status', axis=1, inplace=True)

# Map 'target' according to 'target_values' 
target = target.map(target_values)

In [ ]:
# Transform the remaining columns of cat_data

for i in cat_data:
    cat_data[i] = le.fit_transform(cat_data[i])
target.head()

In [ ]:
cat_data.head()

In [ ]:
# Create new Pandas object 
df = pd.concat([cat_data, num_data, target], axis=1)
df.head()

# **3. Train Data**

In [ ]:
# Create (X, y) Pandas objects for data training 
X = pd.concat([cat_data, num_data], axis=1)
y = target

In [ ]:
# We will use StratifiedShuffleSplit to split the data Taking into consideration that we will get the same ratio on the target column

from sklearn.model_selection import StratifiedShuffleSplit

sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

for train, test in sss.split(X, y):
    X_train, X_test = X.iloc[train], X.iloc[test]
    y_train, y_test = y.iloc[train], y.iloc[test]
    
print('X_train shape', X_train.shape)
print('y_train shape', y_train.shape)
print('X_test shape', X_test.shape)
print('y_test shape', y_test.shape)

# Almost same ratio
print('\nratio of target in y_train :',y_train.value_counts().values/ len(y_train))
print('ratio of target in y_test :',y_test.value_counts().values/ len(y_test))
print('ratio of target in original_data :',df['Loan_Status'].value_counts().values/ len(df))

In [ ]:
# We will use 4 different models for training

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

models = {
    'LogisticRegression': LogisticRegression(random_state=42),
    'KNeighborsClassifier': KNeighborsClassifier(),
    'SVC': SVC(random_state=42),
    'DecisionTreeClassifier': DecisionTreeClassifier(max_depth=1, random_state=42)
}

In [ ]:
# Loss

from sklearn.metrics import precision_score, recall_score, f1_score, log_loss, accuracy_score

def loss(y_true, y_pred, retu=False):
    pre = precision_score(y_true, y_pred)
    rec = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    loss = log_loss(y_true, y_pred)
    acc = accuracy_score(y_true, y_pred)
    
    if retu:
        return pre, rec, f1, loss, acc
    else:
        print('  pre: %.3f\n  rec: %.3f\n  f1: %.3f\n  loss: %.3f\n  acc: %.3f' 
              % (pre, rec, f1, loss, acc))

In [ ]:
# Train data

def train_eval_train(models, X, y):
    for name, model in models.items():
        print(name,':')
        model.fit(X, y)
        loss(y, model.predict(X))
        print('-'*30)

train_eval_train(models, X_train, y_train)

# We can see that best model is LogisticRegression at least for now, SVC is just 
# memorizing the data so it is overfitting.

In [ ]:
# Cross validation

from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=10, random_state=42, shuffle=True)

def train_eval_cross(models, X, y, folds):
    # We will change X & y to dataframe because we will use iloc (iloc don't work on numpy array)
    X = pd.DataFrame(X) 
    y = pd.DataFrame(y)
    idx = [' pre', ' rec', ' f1', ' loss', ' acc']
    for name, model in models.items():
        ls = []
        print(name,':')

        for train, test in folds.split(X, y):
            model.fit(X.iloc[train], y.iloc[train]) 
            y_pred = model.predict(X.iloc[test]) 
            ls.append(loss(y.iloc[test], y_pred, retu=True))
        print(pd.DataFrame(np.array(ls).mean(axis=0), index=idx)[0])  #[0] because we don't want to show the name of the column
        print('-'*30)
        
train_eval_cross(models, X_train, y_train, skf)

# SVC is just memorizing the data, and you can see 
# that here DecisionTreeClassifier is better than LogisticRegression

In [ ]:
# Some explanation about Logistic Regression

x = []
idx = [' pre', ' rec', ' f1', ' loss', ' acc']

# We will use one model
log = LogisticRegression()

for train, test in skf.split(X_train, y_train):
    log.fit(X_train.iloc[train], y_train.iloc[train])
    ls = loss(y_train.iloc[test], log.predict(X_train.iloc[test]), retu=True)
    x.append(ls)
    
# Thats what we get
pd.DataFrame(x, columns=idx)

# (column 0 represent the precision_score of the 10 folds)
# (row 0 represent the (pre, rec, f1, loss, acc) for the first fold)
# then we should find the mean of every column
# pd.DataFrame(x, columns=idx).mean(axis=0)

# **4. Features Engineering**

In [ ]:
# We got it right for most of the features, as you can see we've say at the first of 
# the kernel, that Credit_History and Married etc, are good features, actually 
# Credit_History is the best.

data_corr = pd.concat([X_train, y_train], axis=1)
corr = data_corr.corr()
plt.figure(figsize=(10,7))
sns.heatmap(corr, annot=True);

# Here we got 58% similarity between LoanAmount & ApplicantIncome 
# and that may be bad for our model so we will see what we can do

In [ ]:
# I will try to make some operations on some features, here I just tried diffrent operations on different features,
# having experience in the field, and having knowledge about the data will also help

X_train['new_col'] = X_train['CoapplicantIncome'] / X_train['ApplicantIncome']  
X_train['new_col_2'] = X_train['LoanAmount'] * X_train['Loan_Amount_Term']

In [ ]:
data_corr = pd.concat([X_train, y_train], axis=1)
corr = data_corr.corr()
plt.figure(figsize=(10,7))
sns.heatmap(corr, annot=True);

# new_col 0.03 , new_col_2, 0.047
# Not that much , but that will help us reduce the number of features

In [ ]:
X_train.drop(['CoapplicantIncome', 'ApplicantIncome', 'Loan_Amount_Term', 'LoanAmount'], axis=1, inplace=True)

In [ ]:
train_eval_cross(models, X_train, y_train, skf)

# SVC is improving, but LogisticRegression is overfitting
# We do not change anything so we can see what will happen as we go

In [ ]:
# Let's take a look at the value counts of every label

#print(X_train.shape)
#print(X_train.shape[0]) # Rows
#print(X_train.shape[1]) # Columns

print('************************\n')

for i in range(X_train.shape[1]):
    print(X_train.iloc[:,i].value_counts(), end='\n----------\
--------------------------------------\n')

In [ ]:
# new_col_2

# We can see we got right_skewed
# We can solve this problem with very simple statistical technique , by taking the logarithm of all the values
# because when data is normally distributed that will help improving our model

from scipy.stats import norm

fig, ax = plt.subplots(1,2,figsize=(20,5))

sns.distplot(X_train['new_col_2'], ax=ax[0], fit=norm)
ax[0].set_title('new_col_2 before log')

X_train['new_col_2'] = np.log(X_train['new_col_2'])  # logarithm of all the values

sns.distplot(X_train['new_col_2'], ax=ax[1], fit=norm)
ax[1].set_title('new_col_2 after log');

In [ ]:
# Now we will evaluate our models, and i will do that continuously ,so i don't need to 
# mention that every time

train_eval_cross(models, X_train, y_train, skf)

# Our models improved really good by just doing the previous step.

In [ ]:
# new_col

# Most of our data is 0 , so we will try to change other values to 1

print('before:')
print(X_train['new_col'].value_counts())

X_train['new_col'] = [x if x==0 else 1 for x in X_train['new_col']]
print('-'*50)
print('\nafter:')
print(X_train['new_col'].value_counts())

In [ ]:
train_eval_cross(models, X_train, y_train, skf)

# We are improving our models as we go

In [ ]:
for i in range(X_train.shape[1]):
    print(X_train.iloc[:,i].value_counts(), end='\n------------------------------------------------\n')
    
# Looks better

In [ ]:
# Outliers: we will use boxplot to detect them 

sns.boxplot(X_train['new_col_2']);
plt.title('new_col_2 outliers', fontsize=15);
plt.xlabel('');

In [ ]:
threshold = 0.1  # This number is a hyper parameter, by reducing it, more points are removed.
                 # You can just try different values, the deafult value is (1.5) it works good for most cases.
                 # Be careful, you don't want to try a small number because you may loss some important information from data.
                 # That's why I was surprised when 0.1 gived me the best result
            
new_col_2_out = X_train['new_col_2']

q25, q75 = np.percentile(new_col_2_out, 25), np.percentile(new_col_2_out, 75) # Q25, Q75

print('Quartile 25: {} , Quartile 75: {}'.format(q25, q75))

iqr = q75 - q25
print('iqr: {}'.format(iqr))

cut = iqr * threshold
lower, upper = q25 - cut, q75 + cut
print('Cut Off: {}'.format(cut))
print('Lower: {}'.format(lower))
print('Upper: {}'.format(upper))

outliers = [x for x in new_col_2_out if x < lower or x > upper]
print('Nubers of Outliers: {}'.format(len(outliers)))
print('outliers:{}'.format(outliers))

data_outliers = pd.concat([X_train, y_train], axis=1)
print('\nlen X_train before dropping the outliers', len(data_outliers))
data_outliers = data_outliers.drop(data_outliers[(data_outliers['new_col_2'] > upper) | (data_outliers['new_col_2'] < lower)].index)

print('len X_train before dropping the outliers', len(data_outliers))

In [ ]:
X_train = data_outliers.drop('Loan_Status', axis=1)
y_train = data_outliers['Loan_Status']

In [ ]:
sns.boxplot(X_train['new_col_2']);
plt.title('new_col_2 without outliers', fontsize=15);
plt.xlabel('');

# good :)

In [ ]:
train_eval_cross(models, X_train, y_train, skf)

# Now we get 94.1 ??? for precision & 53.5 for recall

# **Features selection**

In [ ]:
# Self_Employed got really bad corr (-0.00061) , let's try to remove it and see what will happen

data_corr = pd.concat([X_train, y_train], axis=1)
corr = data_corr.corr()
plt.figure(figsize=(10,7))
sns.heatmap(corr, annot=True);

In [ ]:
#X_train.drop(['Self_Employed'], axis=1, inplace=True)

train_eval_cross(models, X_train, y_train, skf)

# looks like Self_Employed is not important
# KNeighborsClassifier improved

# droping all the features Except for Credit_History actually improved KNeighborsClassifier and didn't change anything in other models
# so you can try it by you self
# but don't forget to do that on testing data too

#X_train.drop(['Self_Employed','Dependents', 'new_col_2', 'Education', 'Gender', 'Property_Area','Married', 'new_col'], axis=1, inplace=True)

In [ ]:
data_corr = pd.concat([X_train, y_train], axis=1)
corr = data_corr.corr()
plt.figure(figsize=(10,7))
sns.heatmap(corr, annot=True);

# evaluate the models on Test_data

In [ ]:
X_test.head()

In [ ]:
X_test_new = X_test.copy()

In [ ]:
x = []

X_test_new['new_col'] = X_test_new['CoapplicantIncome'] / X_test_new['ApplicantIncome']  
X_test_new['new_col_2'] = X_test_new['LoanAmount'] * X_test_new['Loan_Amount_Term']
X_test_new.drop(['CoapplicantIncome', 'ApplicantIncome', 'Loan_Amount_Term', 'LoanAmount'], axis=1, inplace=True)

X_test_new['new_col_2'] = np.log(X_test_new['new_col_2'])

X_test_new['new_col'] = [x if x==0 else 1 for x in X_test_new['new_col']]

#X_test_new.drop(['Self_Employed'], axis=1, inplace=True)

# drop all the features Except for Credit_History
#X_test_new.drop(['Self_Employed','Dependents', 'new_col_2', 'Education', 'Gender', 'Property_Area','Married', 'new_col'], axis=1, inplace=True)

In [ ]:
X_test_new.head()

In [ ]:
X_train.head()

In [ ]:
for name,model in models.items():
    print(name, end=':\n')
    loss(y_test, model.predict(X_test_new))
    print('-'*40)